This notebook is currently in progress

# GEE Aster Preprocessing

## Step 1: Create a google earth engine account

Follow this [link](https://code.earthengine.google.com/register) to create a GEE account

## Step 2: Installation


In [ ]:
%pip install git+https://github.com/Mining-for-the-Future/ASTER_preprocessing.git@function_disaggregation

## Step 3: Importing Libraries

In [ ]:
%pip install geemap

importing these modules should trigger the google earth engine authentication process.

Note: in VS Code, the input field for the google earth engine token is at the top of the window, not underneath the cell.

In [ ]:
from ASTER_preprocessing.preprocessing import aster_collection_preprocessing, aster_image_preprocessing
from ASTER_preprocessing.vis_params import vis_params, vis_params_image

In [ ]:
import geemap

## Step 4: Preprocess ASTER image collection using a drawn polygon

In [ ]:
# Create an interactive map
Map = geemap.Map()
Map

In [ ]:
# Get the last drawn feature
geometry = Map.user_roi

In [ ]:
img_dict = aster_collection_preprocessing(geometry, bands = ['B04', 'B3N', 'B02'], masks = ['cloud'])

In [ ]:
img_dict['imagery'].size().getInfo()

## Step 4A: preprocess aster image from drawn point

In [ ]:
Map = geemap.Map()
Map

In [ ]:
geometry = Map.user_roi

### Examine the dates of images intersecting this point to select one

In [ ]:
from ASTER_preprocessing.preprocessing import ee_i

In [ ]:
from datetime import datetime

In [ ]:
collection = ee_i.ImageCollection('ASTER/AST_L1T_003').filterBounds(geometry)

In [ ]:
image_list = collection.toList(collection.size())

In [ ]:
dates = []
for i in range(image_list.length().getInfo()):
    image = ee_i.Image(image_list.get(i))
    date = datetime.utcfromtimestamp(image.get('system:time_start').getInfo()/1000).strftime('%Y-%m-%d')
    dates.append(date)

In [ ]:
dates

In [ ]:
collection.filterDate('2005-03-26', '2005-03-27').size().getInfo()

In [ ]:
image = collection.filterDate('2005-03-26', '2005-03-27').first()

In [ ]:
def draw_map(img):
    map = geemap.Map()
    map.centerObject(img)
    map.addLayer(img)
    return map

In [ ]:
draw_map(image)

In [ ]:
image.bandNames().getInfo()

In [ ]:
image.get('CLOUDCOVER').getInfo()

In [ ]:
from ASTER_preprocessing.data_conversion import aster_dn2toa

In [ ]:
image_toa = aster_dn2toa(image, ['B3N', 'B04', 'B05'])
map = geemap.Map()
map.center_object(image_toa)
map.addLayer(image_toa, vis_params = {'bands': ['B3N', 'B04', 'B05'], 'min': [0, 0, 0], 'max': [1, 1, 1]})
map

In [ ]:
image_proc = aster_image_preprocessing(image, ['B3N', 'B02', 'B01'], masks = ['cloud'])
vis_params = vis_params_image(image_proc, ['B3N', 'B02', 'B01'])

map = geemap.Map()
map.center_object(image_proc)
map.addLayer(image_proc, vis_params = vis_params, name = 'processed_image')
map

In [ ]:
image = aster_image_preprocessing(image, bands = ['B3N', 'B04', 'B05'], masks = [])

In [ ]:
image.bandTypes().getInfo()

In [ ]:
vis_params_image(image, ['B3N', 'B04', 'B05'])

In [ ]:
def vis_params_image(image, bands):
    geometry = image.geometry()
    projection = image.select(bands[0]).projection().getInfo()
    
    percentiles = image.select(bands).reduceRegion(
        reducer = ee_i.Reducer.percentile([5, 95]),
        geometry = geometry,
        crs = projection['crs'],
        crsTransform = projection['transform'],
        bestEffort = True).getInfo()

    b_p5 = [band + '_p5' for band in bands]
    b_p95 = [band + '_p95' for band in bands]
    mins = [percentiles[min] for min in b_p5]
    maxs = [percentiles[max] for max in b_p95]
    vis_params = {'min': mins, 'max': maxs, 'bands': bands}
    return vis_params

In [ ]:
image.geometry().getInfo()

In [ ]:
map = geemap.Map()
map.centerObject(image)
map.addLayer(image)
map

In [ ]:
image.select('B01').projection().getInfo()

## Step 5: Visualize the image collection as a time series

In [ ]:
collection = img_dict['imagery']
first_image = collection.first()
vis_params = vis_params_image(first_image, bands = ['B04', 'B3N', 'B02'])

map2 = geemap.Map()
map2.centerObject(first_image)
# map2.addLayer(first_image, vis_params)
map2.add_time_slider(collection, vis_params = vis_params, time_interval = 1)
map2